

## **OPC Date Selection**

*1. Select information from SQL server*

*2. Download CSV data from SQL server*

*3. Upload CSV data here*



*- Press ctrl+f9 to rull all data. Make sure that current team and file name is accurate -*

**List of current team members**

In [ ]:
current_team = ['Lara Crisp', 'Shianne Elliott', 'Conan Cecconie', 'Jonathan Reale', 'Jacob Friedman', 'Taylor Keyes', 'Beau Thomas']

**The name of the uploaded file should EXACTLY match the one below**

In [ ]:
file_name = 'Program Ledger By Personnel.csv'

**The following cell will ask you for a start date and end date**

In [ ]:
#@title
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

BGV = pd.read_csv(file_name)
BGV['TourDate1'] = pd.to_datetime(BGV['TourDate1'])
BGV.drop_duplicates(subset=['PersonnelNameTitle', 'TourID2'], inplace=True)

print('Please choose a start date (yyyy-mm-dd)')
valid=False
while valid == False:
  start_date = input('Start Date: ')
  if str(start_date) in np.sort(BGV['TourDate1'].astype(str).unique()):
    print('Success!')
    valid=True
  else:
    print('Error: This was either typed incorrectly or the date is not in the uploaded file')

print('Please choose an end date (yyyy-mm-dd)')
valid=False
while valid == False:
  end_date = input('End Date: ')
  if str(end_date) in np.sort(BGV['TourDate1'].astype(str).unique()):
    print('Success!')
    valid=True
  else:
    print('Error: This was either typed incorrectly or the date is not in the uploaded file')

BGV = BGV.loc[(BGV['TourDate1'] >= start_date) & (BGV['TourDate1'] <= end_date)]

opc_team = pd.DataFrame()
for individual in range(len(current_team)):
  opc_team = opc_team.append(BGV.loc[BGV['PersonnelNameTitle'] == str(current_team[individual])+': Tour Booker'])

BGV = BGV.sort_values('TourDate1')
daterange = pd.date_range(start = start_date, end = end_date, freq='d')

opc_team_showed = opc_team.loc[opc_team['TourStatus2'] == 'Showed']

**Tour Counts and 7 Day Average**

In [ ]:
#@title
from matplotlib.ticker import MaxNLocator

for individual in range(len(current_team)):
  name = current_team[individual]
  if name+': Tour Booker' in opc_team['PersonnelNameTitle'].unique():
    fig, ax = plt.subplots(figsize=(15,4))
    
    individual_showed = opc_team_showed.loc[opc_team_showed['PersonnelNameTitle'] == name+': Tour Booker']

    individual_daterange = pd.DataFrame(index=daterange)
    individual_daterange['Tour Count'] = individual_showed['TourDate1'].value_counts()
    individual_daterange['Tour Count'] = individual_daterange['Tour Count'].fillna(0)
    individual_daterange['Tour Date'] = individual_daterange.index.strftime('%Y-%m-%d')
    individual_daterange['Average'] = individual_daterange['Tour Count'].rolling(7).mean()

    individual_daterange.plot('Tour Date', 'Tour Count', kind='bar', color='lightskyblue', alpha=0.75, ax=ax, zorder=3)
    individual_daterange.plot('Tour Date', 'Average', linewidth=3, color='#1f77b4', ax=ax, zorder=5).grid(axis='y', zorder=0, alpha=0.5)

    count = individual_daterange['Tour Count'].sum().astype(int)
    average = individual_daterange['Tour Count'].fillna(0).mean()
    plt.axhline(y=average, color='lightcoral', zorder=4)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    # Format x-axis based on length of date range
    if len(daterange) >= 182:
      [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 7 != 0]
    elif len(daterange) >= 62:
      [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 4 != 0]
    elif len(daterange) >= 31:
      [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 2 != 0]


    plt.xlabel('')
    plt.xticks(rotation=90)
    plt.title('   '+str(name)+' (Total: '+str(count)+')', weight='bold', loc='left', fontsize=12)
    plt.title('Tour Count Average from '+str(start_date)+' to '+str(end_date)+'   ', loc='right', fontsize=12)
    plt.legend(['7 Day Average', 'Overall Average: '+str(round(average,2)), 'Daily Tour Count'])
  else:
    print('Cannot find data for ' + name)

In [ ]:
#@title
from matplotlib.ticker import MaxNLocator

for individual in range(len(current_team)):
  name = current_team[individual]
  if name+': Tour Booker' in opc_team['PersonnelNameTitle'].unique():
    fig, ax = plt.subplots(figsize=(15,4))
    
    individual_showed = opc_team_showed.loc[opc_team_showed['PersonnelNameTitle'] == name+': Tour Booker']

    df1 = individual_showed.loc[(individual_showed['ContractStatus1'] != 'Active\r\nActive') & (individual_showed['ContractStatus1'] != 'Suspense\r\nSuspense')]

    df2 = individual_showed.loc[individual_showed['CommissionVolume'] > 0]
    df2 = df2.loc[(df2['ContractStatus1'] == 'Active\r\nActive') | (df2['ContractStatus1'] == 'Suspense\r\nSuspense')]

    individual_daterange = pd.DataFrame(index=daterange)
    individual_daterange['Tour Count'] = individual_showed['TourDate1'].value_counts()
    individual_daterange['Tour Count'] = individual_daterange['Tour Count'].fillna(0)
    individual_daterange['Tour Date'] = individual_daterange.index.strftime('%Y-%m-%d')
    individual_daterange['Average'] = individual_daterange['Tour Count'].rolling(7).mean()

    individual_daterange['Purchase Count'] = df2['TourDate1'].value_counts()
    individual_daterange['Shown Count'] = df1['TourDate1'].value_counts()
    individual_daterange['Purchase Count'] = individual_daterange['Purchase Count'].fillna(0)
    individual_daterange['Shown Count'] = individual_daterange['Shown Count'].fillna(0)

    individual_daterange.plot('Tour Date', 'Tour Count', kind='bar', color='lightskyblue', alpha=0.75, ax=ax, zorder=3)
    individual_daterange.plot('Tour Date', 'Purchase Count', kind='bar', color='forestgreen', ax=ax, alpha=0.5, zorder=5)
    individual_daterange.plot('Tour Date', 'Average', linewidth=3, color='#1f77b4', ax=ax, zorder=6).grid(axis='y', zorder=0, alpha=0.5)

    count = individual_daterange['Tour Count'].sum().astype(int)
    average = individual_daterange['Tour Count'].fillna(0).mean()
    plt.axhline(y=average, color='lightcoral', zorder=4)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    # Format x-axis based on length of date range
    if len(daterange) >= 182:
      [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 7 != 0]
    elif len(daterange) >= 62:
      [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 4 != 0]
    elif len(daterange) >= 31:
      [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 2 != 0]


    plt.xlabel('')
    plt.xticks(rotation=90)
    plt.title('   '+str(name)+' (Total: '+str(count)+')', weight='bold', loc='left', fontsize=12)
    plt.title('Tour Count Average from '+str(start_date)+' to '+str(end_date)+'   ', loc='right', fontsize=12)
    plt.legend(['7 Day Average', 'Overall Average: '+str(round(average,2))])
  else:
    print('Cannot find data for ' + name)